In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf

Read Data

In [ ]:
df = pd.read_csv("train.csv")
df.head(10)

In [ ]:
df.columns

In [ ]:
for column in df.columns:
  print(df[column].unique())
  print(" ")
df.drop_duplicates(inplace=True)
len(df)

Delete rows that contain invalid values such as NaN, -1, missing, unspecified, etc

In [ ]:
def CleanData(df, col):
        del_index=df[
        (df[col]=='unspecified') | 
        (df[col]=='missing') | 
        (df[col]=='not-recorded') | 
        (df[col]=='not-available') |
        (df[col]=='-1') |
        (df[col]=='unknown') |
        (df[col]=='na') |
        (df[col]=='unestablished') | 
        (df[col]== np.nan) ].index
        df.drop(del_index, inplace=True)
        df.reset_index()    
        return df
col_clean=['Vehicle Class', 'Engine Size(L)', 'Cylinders', 'Transmission', 'Fuel Type']
for col in col_clean:
        df=CleanData(df, col)
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)
len(df)

Double check for null values

In [ ]:
np.sum(df.isnull())

In [ ]:
for column in df.columns:
  print(df[column].unique())
  print(" ")

Method to find unique units of fuel consumption

In [ ]:
def is_float(num):
    try:
        float(num)
        return True
    except:
        return False
list_temp=[]

for record in df["Fuel Consumption City"]:
    try:
        list_word=record.split(" ", 1)
        list_temp.append(list_word[1])
    except:
        continue
for rec in set(list_temp):
    if(is_float(rec)):
        continue
    else:
        print(rec)

Convert every data unit into the same unit, L/100km

In [ ]:
def convert_to_l_per_hundred_km(df):
    for col in ["Fuel Consumption City", "Fuel Consumption Hwy", "Fuel Consumption Comb"]:
        list_temp=[]
        for record in df[col]:
            try:
                list_word=record.split(" ", 1)
                if list_word[1]=="mpg Imp.":
                    list_temp.append(282.481/float(list_word[0]))
                elif list_word[1]=="liters per 100 km" or list_word[1]=="L/100 km" or list_word[1]=="L/100km":
                    list_temp.append(float(list_word[0]))
                elif list_word[1]=="km/L" or list_word[1]=="km per L":
                    list_temp.append(100/float(list_word[0]))
                elif list_word[1]=="L/10km":
                    list_temp.append(float(list_word[0])*10)
                elif list_word[1]=="MPG (AS)":
                    list_temp.append(235.214/float(list_word[0]))
            except:
                list_temp.append(0)
        df[col] = list_temp
    return df
    

In [ ]:
df=convert_to_l_per_hundred_km(df)

In [ ]:
df.head(10)

Remove zero values

In [ ]:
def remove_zeros(df, column_name=""):
  df = df[df[column_name] != 0]
  df.reset_index()
  return df

Delete outliers and zero values

In [ ]:
#Removing Outliers
def remove_outliers(df, column_name=""):
  q1 = df[column_name].quantile(0.25)
  q3 = df[column_name].quantile(0.75)

  IQR = q3 - q1

  lower_limit = q1 - 1.5 * IQR
  upper_limit = q3 + 1.5 * IQR

  df = df[(df[column_name] >= lower_limit) & (df[column_name] <= upper_limit)]
  df.reset_index()
  df=remove_zeros(df, column_name)
  return df


In [ ]:
df = remove_outliers(df,"Fuel Consumption City")
df = remove_outliers(df,"Fuel Consumption Hwy")
df = remove_outliers(df,"Fuel Consumption Comb")

In [ ]:
df[["Fuel Consumption City", "Fuel Consumption Hwy", "Fuel Consumption Comb", "CO2 Emissions(g/km)"]].describe()


In [ ]:
# replace_value=[]
# columns=['Make', 'Vehicle Class', 'Engine Size(L)', 'Cylinders', 'Transmission',
#        'Fuel Type', 'Fuel Consumption City', 'Fuel Consumption Hwy',
#        'Fuel Consumption Comb']
# for col in columns:
#     try:
#         replace_value.append(np.mean(df[col]))
#     except:
#         print(col)
#         replace_value.append(df[col].mode()[0])
# print(replace_value)

In [ ]:
df.dtypes

Manual imputation

In [ ]:
# df=pd.read_csv('train.csv')
# for col in ["Fuel Consumption City", "Fuel Consumption Hwy", "Fuel Consumption Comb"]:
#     list_temp=[]
#     for record in df[col]:
#         try:
#             list_word=record.split(" ", 1)
#             list_temp.append(convert_to_l_per_hundred_km(list_word[0], list_word[1]))
#         except:
#             list_temp.append(0)
#     df[col] = list_temp

# def FillData(df, col, value_for_replace):
#         wrong_index=df[
#         (df[col]=='unspecified') | 
#         (df[col]=='missing') | 
#         (df[col]=='not-recorded') | 
#         (df[col]=='not-available') |
#         (df[col]=='-1') |
#         (df[col]=='unknown') |
#         (df[col]=='na') |
#         (df[col]=='unestablished') |
#         (df[col]=='zero')].index
#         df.loc[wrong_index, col]=value_for_replace
#         df[col].fillna(value_for_replace, inplace=True)
#         print(df[col].unique())
#         return df
# col_clean=['Make', 'Vehicle Class', 'Engine Size(L)', 'Cylinders', 'Transmission',
#        'Fuel Type', 'Fuel Consumption City', 'Fuel Consumption Hwy',
#        'Fuel Consumption Comb']
# for i in range(len(col_clean)):
#         df=FillData(df, col_clean[i], replace_value[i])
# for column in df.columns:
#   print(df[column].unique())
#   print(" ")
# df.drop_duplicates(inplace=True)
# len(df)

In [ ]:
np.sum(df.isnull())

In [ ]:
df = remove_outliers(df,"Fuel Consumption City")
df = remove_outliers(df,"Fuel Consumption Hwy")
df = remove_outliers(df,"Fuel Consumption Comb")
df = remove_outliers(df,"CO2 Emissions(g/km)")

In [ ]:
df.describe()

Method to sort qualitative data based on their CO2 Consumption mean

In [ ]:
def visualize_sorted_mean(df, column_name):
  cols=df[column_name].unique()
  avg=[np.average(df["CO2 Emissions(g/km)"][df[column_name]==col]) for col in cols]
  median=[np.median(df["CO2 Emissions(g/km)"][df[column_name]==col]) for col in cols]
  dict_col=dict(zip(cols, zip(avg, median)))
  dict_col=sorted(dict_col.items(), key=lambda item: item[1])

  fig, ax = plt.subplots(figsize=(7,10))
  bar_width = 0.4

  bar1 = ax.barh(np.arange(len(cols)), [dict_col[:][i][1][0] for i in range(len(dict_col))], height=bar_width, color='yellow', alpha=0.5, label='Average')
  bar2 = ax.barh(np.arange(len(cols)) + bar_width, [dict_col[:][i][1][1] for i in range(len(dict_col))], height=bar_width, color='orange', alpha=0.5, label='Median')

  ax.set_yticks(np.arange(len(cols)) + bar_width / 2)
  ax.set_yticklabels(dict_col)
  ax.set_xlabel('CO2 Emissions(g/km)')
  ax.set_ylabel(column_name)
  ax.set_title('Visualization of CO2 emissions by ' + column_name)
  ax.legend()

  plt.show()

def generate_dict(df, column_name):
  cols=df[column_name].unique()
  avg=[np.average(df["CO2 Emissions(g/km)"][df[column_name]==col]) for col in cols]
  median=[np.median(df["CO2 Emissions(g/km)"][df[column_name]==col]) for col in cols]
  dict_col=dict(zip(cols, zip(avg, median)))
  dict_col=sorted(dict_col.items(), key=lambda item: item[1])
  return dict_col, avg

In [ ]:
visualize_sorted_mean(df, "Make")

In [ ]:
visualize_sorted_mean(df, "Vehicle Class")

Method to add new column (Average CO2 Consumption of each unique label)

In [ ]:
def AddAvgColumn(df, column_name):
    make_map={}
    dict_col, avg=generate_dict(df, column_name)
    for i in range(len(dict_col)):
        make_map[dict_col[i][0]]=avg[i]
    new_col=column_name+"Avg"
    df[new_col]=df[column_name].map(make_map)
    # df[new_col] = (df[new_col]-df[new_col].min())/df[new_col].max()
    # df[new_col] = (df[new_col]-192.6185147143735)/271.26871156214463
    
    print(make_map)
    return df

In [ ]:
df=AddAvgColumn(df, "Make")
df=AddAvgColumn(df, "Vehicle Class")
df=AddAvgColumn(df, "Transmission")
df=AddAvgColumn(df, "Fuel Type")

In [ ]:
df.head(5)

In [ ]:
print(df['MakeAvg'].unique())

In [ ]:
visualize_sorted_mean(df, "Transmission")


In [ ]:
col_box=['Fuel Consumption City', 'Fuel Consumption Hwy','Fuel Consumption Comb', "CO2 Emissions(g/km)"]
plt.subplots(figsize=(10,10))
plt.delaxes()
i=1
for col in col_box:
    plt.subplot(2,2,i)
    plt.boxplot(df[col])
    plt.title(col)
    i+=1
plt.show()

In [ ]:
#['X' 'E' 'Z' nan 'D' '-1' 'missing' 'unspecified' 'not-recorded' 'unknown', 'unestablished' 'na' 'not-available' 'N']
# len(df[df["Fuel Type"=="unknown"]])
df["Fuel Type"].value_counts()

In [ ]:
# corr=df[["Fuel Consumption City", "Fuel Consumption Hwy", "Fuel Consumption Comb", "CO2 Emissions(g/km)", "MakeAvg","Make",  "Cylinders", "Engine Size(L)"]].corr()
corr=df[['Cylinders', 'Fuel Consumption City', 'Fuel Consumption Hwy',
       'Fuel Consumption Comb', 'MakeAvg',
       'Vehicle ClassAvg', 'TransmissionAvg', 'Fuel TypeAvg', "CO2 Emissions(g/km)"]].corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

In [ ]:
df["Engine/Cylinders"]=df["Engine Size(L)"].astype(float)/df["Cylinders"].astype(float)
corr=df[["CO2 Emissions(g/km)", "Engine/Cylinders"]].corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

In [ ]:
#ALT 1
# index_del=df[(df["Fuel Consumption City"].astype(float)<100) | (df["Fuel Consumption Hwy"].astype(float)<100) | (df["Fuel Consumption Comb"].astype(float)<100)].index
# df.drop(index_del, inplace=True)

In [ ]:
#ALT 2
df["Fuel Consumption City"]=np.log10(df["Fuel Consumption City"])
df["Fuel Consumption Hwy"]=np.log10(df["Fuel Consumption Hwy"])
df["Fuel Consumption Comb"]=np.log10(df["Fuel Consumption Comb"])

In [ ]:
plt.scatter(df["Fuel Consumption City"], df["CO2 Emissions(g/km)"])
plt.xlabel("Fuel Consumption City")
plt.ylabel("CO2 Emissions(g/km)")  
plt.show()

plt.scatter(df["Fuel Consumption Hwy"], df["CO2 Emissions(g/km)"])
plt.xlabel("Fuel Consumption Hwy")
plt.ylabel("CO2 Emissions(g/km)")  
plt.show()

plt.scatter(df["Fuel Consumption Comb"], df["CO2 Emissions(g/km)"])
plt.xlabel("Fuel Consumption Comb")
plt.ylabel("CO2 Emissions(g/km)")  
plt.show()

plt.scatter(df["Cylinders"], df["CO2 Emissions(g/km)"])
plt.xlabel("Cylinders")
plt.ylabel("CO2 Emissions(g/km)")  
plt.show()
# corr=df[["Fuel Consumption City", "Fuel Consumption Hwy", "Fuel Consumption Comb", "CO2 Emissions(g/km)", "Make", "Cylinders", "Engine Size(L)"]].corr()

In [ ]:
# del_index=df[(df["Fuel Consumption City"]>15000) | (df["Fuel Consumption Hwy"]>15000) | (df["Fuel Consumption Comb"]>800)].index
del_index=df[(df["Fuel Consumption City"]>15000000)].index
df.drop(del_index, inplace=True)
df.reset_index()
plt.scatter(df["Fuel Consumption City"], df["Fuel Consumption Hwy"])
plt.xlabel("Fuel Consumption City")
plt.ylabel("Fuel Consumption Hwy")  
plt.show()

In [ ]:
plt.scatter(df["Engine Size(L)"], df["CO2 Emissions(g/km)"])
plt.xlabel("Fuel Consumption City")
plt.ylabel("CO2 Emissions(g/km)")  
plt.show()

plt.scatter(df["Cylinders"], df["CO2 Emissions(g/km)"])
plt.xlabel("Fuel Consumption Hwy")
plt.ylabel("CO2 Emissions(g/km)")  
plt.show()
# corr=df[["Fuel Consumption City", "Fuel Consumption Hwy", "Fuel Consumption Comb", "CO2 Emissions(g/km)", "Make", "Cylinders", "Engine Size(L)"]].corr()

In [ ]:
print(len(df))
df.to_csv("preprocessing.csv")

In [ ]:
df.head(20)

In [ ]:
len(df)

In [ ]:
df_train = pd.read_csv("preprocessing.csv", index_col=0)
df_train.head(10)

In [ ]:
df_train.columns

In [667]:
tf.keras.backend.clear_session()
model = tf.keras.Sequential([
    tf.keras.layers.Dense(6, activation="relu"),
    tf.keras.layers.Dense(8, activation="relu"),
    tf.keras.layers.Dense(6, activation="relu"),
    tf.keras.layers.Dense(1, activation="linear")
])
# col_train=['Make', 'Vehicle Class', 'Engine Size(L)', 'Cylinders', 'Transmission',
#        'Fuel Type', 'Fuel Consumption City', 'Fuel Consumption Hwy',
#        'Fuel Consumption Comb', 'CO2 Emissions(g/km)', 'MakeAvg',
#        'Vehicle ClassAvg', 'TransmissionAvg', 'Fuel TypeAvg']
col_train=['Cylinders',
       'Fuel Consumption City', 'Fuel Consumption Hwy',
       'Fuel Consumption Comb']

# model.compile(loss="mae", optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), metrics=tf.keras.metrics.RootMeanSquaredError())
model.compile(loss="mse", optimizer=tf.keras.optimizers.Adam(3e-4), metrics=tf.keras.metrics.RootMeanSquaredError())

history=model.fit(df_train[col_train], df_train["CO2 Emissions(g/km)"], epochs=50, validation_split=0.2, shuffle=False)

841/841 [==============================] - 7s 9ms/step - loss: 1004.4962 - root_mean_squared_error: 31.6938 - val_loss: 979.2927 - val_root_mean_squared_error: 31.2937
Epoch 18/50
841/841 [==============================] - 7s 8ms/step - loss: 1000.6661 - root_mean_squared_error: 31.6333 - val_loss: 975.5688 - val_root_mean_squared_error: 31.2341
Epoch 19/50
841/841 [==============================] - 6s 7ms/step - loss: 996.8600 - root_mean_squared_error: 31.5731 - val_loss: 971.8648 - val_root_mean_squared_error: 31.1747
Epoch 20/50
841/841 [==============================] - 8s 9ms/step - loss: 993.0740 - root_mean_squared_error: 31.5131 - val_loss: 968.1804 - val_root_mean_squared_error: 31.1156
Epoch 21/50
841/841 [==============================] - 6s 7ms/step - loss: 989.3071 - root_mean_squared_error: 31.4533 - val_loss: 964.5140 - val_root_mean_squared_error: 31.0566
Epoch 22/50
841/841 [==============================] - 7s 9ms/step - loss: 985.5591 - root_mean_squared_error: 31.3

KeyboardInterrupt: 

In [ ]:
loss = np.sqrt(history.history['loss'])
val_loss = np.sqrt(history.history['val_loss'])

epochs = range(len(loss))

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
df_test = pd.read_csv("test.csv")
df_test=convert_to_l_per_hundred_km(df_test)
df_test.head()

In [ ]:
for col in df_test.columns:
    nan_index=df_test[(df_test[col]=='unspecified') | 
        (df_test[col]=='missing') | 
        (df_test[col]=='not-recorded') | 
        (df_test[col]=='not-available') |
        (df_test[col]=='-1') |
        (df_test[col]=='unknown') |
        (df_test[col]=='na') |
        (df_test[col]=='unestablished') | 
        (df_test[col]=='zero') | 
        (df_test[col]==-9999) | 
        (df_test[col]==-1) | 
        (df_test[col]== 0) | 
        (df_test[col]== np.nan) ].index
    df_test.loc[nan_index,col] = np.nan
df_checknull=df_test[["Id", 'Vehicle Class', 'Engine Size(L)', 'Cylinders', 'Transmission', 'Fuel Type', 'Fuel Consumption City', 'Fuel Consumption Hwy', 'Fuel Consumption Comb']].isnull()
df_checknull.drop_duplicates(inplace=True)
print(len(df_checknull))
# print(df_checknull.drop_duplicates())
df_checknull.to_excel("Combination NULL.xlsx")
nan_counts = df_test[["Id", 'Vehicle Class', 'Engine Size(L)', 'Cylinders', 'Transmission', 'Fuel Type']].isna().sum(axis=1)

# Filter rows with more than 3 NaN values
# result = df[nan_counts > 3]
# print(result)
     

In [ ]:
prediction=model.predict(np.expand_dims(np.asarray(df_test[col_train]), -1).astype(np.float32))

In [ ]:
df_output=pd.DataFrame(prediction)
df_output.to_excel("output.xlsx")